In [1]:
%load_ext autotime
import warnings; warnings.filterwarnings('ignore')

In [9]:
import numpy as np, pandas as pd, geopandas as gpd
import matplotlib.pyplot as plt
import rasterio
from rasterio import features
from sklearn.ensemble import RandomForestClassifier

time: 2.96 ms


In [3]:
img = rasterio.open('/mnt/ext1/data/sur/lulc/mosaic.tif').read()
roi = rasterio.open('train.tif').read(1)
X = img[:, roi>0]
y = roi[roi>0]

time: 1min 20s


In [4]:
rf = RandomForestClassifier(n_estimators=10, n_jobs=3, oob_score=True)
rf.fit(X.T, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=3,
            oob_score=True, random_state=None, verbose=0, warm_start=False)

time: 10.8 s


In [5]:
print 'Our oob prediction accuracy is ' +  str(100*rf.oob_score_ )[:6] + '%'
bands = [1, 2, 3]
for b, imp in zip(bands, rf.feature_importances_):
    print('Band {b} importance: {imp}'.format(b=b, imp=imp))

Our oob prediction accuracy is 95.917%
Band 1 importance: 0.285806353927
Band 2 importance: 0.31424434327
Band 3 importance: 0.399949302803
time: 123 ms


In [6]:
df = pd.DataFrame()
df['truth'] = y
df['predict'] = rf.predict(X.T)
print(pd.crosstab(df.truth, df.predict, margins=True))

predict       1      2     3        4      5      All
truth                                                
1        119737      0     4       20      0   119761
2             2  18764     4      343      5    19118
3             0     17  5781       88      3     5889
4             1     67     5  1078818    149  1079040
5             1     48    13     4364  49297    53723
All      119741  18896  5807  1083633  49454  1277531
time: 1.25 s


In [8]:
del X, y

time: 2.92 ms


In [7]:
tshape = img.shape[1:2]
shape = (img.shape[0], img.shape[1]*img.shape[2])
img = img.reshape(shape)
predict_class = rf.predict(img.T).reshape(tshape)

MemoryError: 

time: 1.19 s
